In [1]:
from pathlib import Path
from utils import QmdlLogsHelper
import numpy as np
import pandas as pd

In [2]:
path_project = Path('/disk/sean/5glogs/')
path_data = Path('data.hangover/trip_1.forward/')
path_rawlogs_list = [path_project / path_data / Path(f'qmdl_{i}.qmdl') for i in range(1, 21)]
# [PosixPath('/disk/sean/5glogs/data.hangover/trip_1.forward/qmdl_1.qmdl'),
#  PosixPath('/disk/sean/5glogs/data.hangover/trip_1.forward/qmdl_2.qmdl'),
#  ...
#  PosixPath('/disk/sean/5glogs/data.hangover/trip_1.forward/qmdl_20.qmdl')]

In [3]:
logs_helper = QmdlLogsHelper(path_rawlogs_list[0:])

In [4]:
logs_helper.df_logs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23498166 entries, 0 to 23498165
Data columns (total 3 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   log             object
 1   log_len         int64 
 2   hangover_start  int64 
dtypes: int64(2), object(1)
memory usage: 537.8+ MB


In [7]:
df_hostart = logs_helper.df_logs[ logs_helper.df_logs.hangover_start == 1]
df_hostart.describe()

,log_len,hangover_start
count,185.000000,185.0
mean,260.189189,1.0
std,328.606498,0.0
min,9.000000,1.0
25%,68.000000,1.0
50%,129.000000,1.0
75%,281.000000,1.0
max,1978.000000,1.0


In [34]:
idx = 452927
logs_helper.df_logs.loc[(idx-10):idx]

,log,log_len,hangover_start
452917,b'\x99\x02\x14\x00O\xb9+`\xa2\xf5\x05\x01\x917...,24,0
452918,b'\x99\x02\x15\x00?\xbc+`\xa2\xf5\x05\x01\x108...,25,0
452919,b'\x99\x02\x14\x00\xd6\xbc+`\xa2\xf5\x05\x01\x...,22,0
452920,b'\xb8',1,0
452921,b'\x99\x02\x12\x00\xd2\xbc+`\xa2\xf5\x05\x01\x...,22,0
452922,b'\x99\x02\x12\x00I\xbd+`\xa2\xf5\x05\x01\x96\...,22,0
452923,b'\x10\x00I\x00I\x00D\x15\x1e\xbf+`\xa2\xf5\x0...,79,0
452924,b'\x99\x02\x11\x009\xbf+`\xa2\xf5\x05\x01\x97\...,21,0
452925,"b'\x99\x02\x15\x00\xe1\x00,`\xa2\xf5\x05\x01\x...",25,0
452926,"b'\x99\x02\x14\x00r\x01,`\xa2\xf5\x05\x01\x917...",24,0


In [5]:
df_list = []
for qmdl_path in path_rawlogs_list[0:2]:
    print(qmdl_path)
    with qmdl_path.open("rb") as f:
        logs = f.read()
        logs = logs.replace(b'\x7d\x5d', b'\x7d') # 7D5D -> 7D
        logs = logs.replace(b'\x7d\x5e', b'\x7e') # 7D5E -> 7E
        logs_list = logs.split(b'\x7e')
    print(len(logs_list), logs_list[0])
    df = pd.DataFrame({'log': logs_list, 'hangover_start': [0]*len(logs_list)})
    df_list.append(df)
df = pd.concat(df_list, ignore_index=True)
df

/disk/sean/5glogs/data.hangover/trip_1.forward/qmdl_1.qmdl
129299 b"\x00\x07\x01r\x0e\xec'\x81"
/disk/sean/5glogs/data.hangover/trip_1.forward/qmdl_2.qmdl
1154839 b'K\xe7w\x94\xa6h\x85\xb6m\xb1\xa6`\xc0\xf6`h\x06Z\x8a\xc5mZWuM7js\xd7hDwi\x11&u\xd3\x96h\x077v\xb8\xc6m\x8bV'


,log,hangover_start
0,"b""\x00\x07\x01r\x0e\xec'\x81""",0
1,b'$\xe2\xc7\xefp\x1a\x1a\x87O\x7f\xfa\xe8\x87\...,0
2,"b'\xfe\x1a\xe7\x04r\n""GN\x7f\xe2\xfa\x07\xe2p\...",0
3,"b""*\x1a\x07.}\xf8\xd0'Ls&\xfe\xa7\xa1\x7f\xfe\...",0
4,b'\x02\xceG\xe1p\x0e\xfe\x07\xa4}\xd6\xd8\x07!',0
...,...,...
1284133,b'\xeb\xb7qV\xf7t',0
1284134,"b""\xd7h\xe0\x07}\xa3wp\xd3\x87\x83l\xa8}\xda\x...",0
1284135,"b""\xc2'}j\xb7w\xedW\x83\xef\xc7rE\xd7\x87\xa5\...",0
1284136,b'\xf3\x97q\x8e\xa6r\xf9\x17z\x83\xc7z\xfa\xc7...,0


In [66]:
# hangover start logs
for i, log in enumerate(df['log']):
    if log[0:1] == b'\x60' and (a := b'\x74') in log:
        id1 = log.index(a)
        if (id2 := id1 + 1) < len(log) and (nxt := log[id2] & b'\x0f'[0]) == b'\x0c'[0]:
            df.at[i, 'hangover_start'] = 1
            print(i, id1, nxt)

452927 57 12
503026 23 12
503457 23 12
504234 23 12
571058 8 12
572029 8 12
573186 8 12
640216 68 12
641973 5 12
1064429 195 12
1261652 33 12


In [51]:
df[df.hangover_start == 1]

,log,hangover_start
452927,"b'`=\x00f""(\x02\xfc\\\xa2\xf5\x05\x01\x00Z\xe2...",1
503026,"b'`\xf8\x8e\xaf\x17k\x1fW},\x87|\xd0W\x8f\x1fX...",1
503457,"b'`\xf8\x8e\xaf\x17k\x1fW},\x87|\xd0W\x8f\x1fX...",1
504234,"b'`\xf8\x8e\xaf\x17k\x1fW},\x87|\xd0W\x8f\x1fX...",1
571058,"b'`w|\x8a\xa7\x85\x1d\x17t,H'",1
572029,"b'`w|\x8a\xa7\x85\x1d\x17t,H'",1
573186,"b'`w|\x8a\xa7\x85\x1d\x17t,H'",1
640216,"b'`I\x00f""C\x8d\xc1o\xa2\xf5\x05\x01\x00Z\xe2\...",1
641973,b'`\xe9{\xee\xdatLH\xbf\x0bi\xb3x\xb2\x85\x87\...,1
1064429,b'`\xe7\x03H\xb5\x87U\xb2\xc5\x8a\x9a\x15F\x90...,1


In [53]:
len(logs_helper.logs_list), logs_helper.logs_list[0]

(1284138, b"\x00\x07\x01r\x0e\xec'\x81")

In [54]:
log = logs_helper.logs_list[0]; print(log)
log_uint = np.frombuffer(log, dtype=np.uint8); print(log_uint)
log_float = log_uint / 255; print(log_float)

b"\x00\x07\x01r\x0e\xec'\x81"
[  0   7   1 114  14 236  39 129]
[0.         0.02745098 0.00392157 0.44705882 0.05490196 0.9254902
 0.15294118 0.50588235]


In [39]:
logs_uint8_list = [np.frombuffer(log, dtype=np.uint8) for log in logs_helper.logs_list]

In [40]:
len(logs_uint8_list), logs_uint8_list[0]

(23498166, array([  0,   7,   1, 114,  14, 236,  39, 129], dtype=uint8))

In [ ]:
logs_float_list = [log/255 for log in logs_uint8_list]

In [27]:
len(logs_float_list), logs_float_list[0]

(23498166,
 array([ 0.        ,  0.02745098,  0.00392157,  0.44705882,  0.05490196,
        -0.07843137,  0.15294118, -0.49803922]))

In [19]:
LOG_LENGTH = 2560
for i, log in enumerate(logs_uint8_list[:5]):
    logs_float = np.zeros([LOG_LENGTH,], dtype=np.float32)
    if len(log) > LOG_LENGTH:
        logs_float[:LOG_LENGTH] = log[:LOG_LENGTH]
    else:
        logs_float[:len(log)] = log
    logs_float /= 255.0
    print(logs_float.shape, logs_float)

(2560,) [0.         0.02745098 0.00392157 ... 0.         0.         0.        ]
(2560,) [ 0.14117648 -0.11764706 -0.22352941 ...  0.          0.
  0.        ]
(2560,) [-0.00784314  0.10196079 -0.09803922 ...  0.          0.
  0.        ]
(2560,) [0.16470589 0.10196079 0.02745098 ... 0.         0.         0.        ]
(2560,) [ 0.00784314 -0.19607843  0.2784314  ...  0.          0.
  0.        ]


In [17]:
data = logs_helper.get_dataset_ho()
print(data, data.shape)

[[282404 282504      1]
 [505972 506072      1]
 [522872 522972      1]
 [736911 737011      1]
 [882025 882125      1]
 [ 39900  40000      0]
 [ 79900  80000      0]
 [119900 120000      0]
 [159900 160000      0]
 [199900 200000      0]] (10, 3)


In [15]:
len(max(logs_helper.logs_list, key=len)) # 5402
max([len(l) for l in logs_helper.logs_list]) # 5402

5402

In [16]:
np.vstack([[[1], [2]], [[3],[4]]])

array([[1],
       [2],
       [3],
       [4]])

In [20]:
# len(logs_helper.logs_list) # qmdl_1.qmdl: 129299 = 72 (modem events) + 129227 (non-modem events)
# sum([1 if log[0:1] == b'\x60' else 0 for log in logs_helper.logs_list]) # 72 (modem events)
# print(b'\x60') # b'`'

In [56]:
modem_event_logs = [(i, log) for i, log in enumerate(logs_helper.logs_list) if log[0:1] == b'\x60']
non_modem_event_logs = [(i, log) for i, log in enumerate(logs_helper.logs_list) if log[0:1] != b'\x60']
a = len(modem_event_logs); b = len(non_modem_event_logs); c = len(logs_helper.logs_list)
print(f'{a:10,d} events modem\n{b:10,d} events non_modem\n{c:10,d} events all'); assert a + b == c
# modem_event_logs[0][:10] # b"`\x02\xe7Np6\xdc'\xaf}"
# non_modem_event_logs[0][:10] # b"\x00\x07\x01r\x0e\xec'\x81"

       565 events modem
 1,283,573 events non_modem
 1,284,138 events all


In [57]:
modem_event_logs_length_list = [len(log) for i, log in modem_event_logs]
print(f'max_length (modem_event_logs): {max(modem_event_logs_length_list)}')
print(modem_event_logs_length_list[:20])

max_length (modem_event_logs): 1514
[19, 53, 38, 35, 4, 15, 24, 284, 34, 19, 28, 93, 17, 1514, 4, 9, 44, 43, 19, 113]


In [58]:
non_modem_event_logs_length_list = [len(l) for i, l in non_modem_event_logs]
print(f'max_length (non_modem_event_logs): {max(non_modem_event_logs_length_list)}') # 4937, 8250, 8258

max_length (non_modem_event_logs): 8250


In [59]:
a = b'\x74'
b = b'\xFF'
assert a[0] == a[0] & b[0] # 116
assert a == bytes([a[0] & b[0]]) # b't'
assert b'\x0f'[0] == 15

In [60]:
# hangover start logs
ho_start_logs = []
for i, log in modem_event_logs:
    if (a := b'\x74') in log:
        id1 = log.index(a)
        if (id2 := id1 + 1) < len(log) and (nxt := log[id2] & b'\x0f'[0]) == b'\x0c'[0]:
            print(i, id1, nxt)
            ho_start_logs.append((i, log))

452927 57 12
503026 23 12
503457 23 12
504234 23 12
571058 8 12
572029 8 12
573186 8 12
640216 68 12
641973 5 12
1064429 195 12
1261652 33 12


In [61]:
print(len(ho_start_logs), ho_start_logs[0])

11 (452927, b'`=\x00f"(\x02\xfc\\\xa2\xf5\x05\x01\x00Z\xe2\x0f\xfc\x04\x00\x00\x00\x00\x9b\xa1\x10\xfc\x00\x07\x85\x12\xfc]\xe2b\xfc\n \x01\x01\xff\xff\xe8\x03\x052\x00Y\xe2q\xfc\x03\xe1\n\x01\x06\xa5t\xfc\x02\x08\x85\xb8\xfc\x81\xe2')


In [62]:
for i, log in enumerate(logs_helper.logs_list):
    if log[0:1] == b'\x60' and (a := b'\x74') in log:
        idx = log.index(a)
        if idx + 1 < len(log):
            nxt = log[idx + 1] & b'\x0f'[0]
            if nxt == b'\x0c'[0]:
                print(i, idx, nxt)

452927 57 12
503026 23 12
503457 23 12
504234 23 12
571058 8 12
572029 8 12
573186 8 12
640216 68 12
641973 5 12
1064429 195 12
1261652 33 12


In [16]:
ho_start_ids = [64, 118, 127, 196, 238]
ho_start_logs = [modem_event_logs[i] for i in ho_start_ids]
[len(l) for l in ho_start_logs]

[88, 142, 163, 33, 153]

In [98]:
# hangover success
for i, log in enumerate(modem_event_logs):
    if (a := b'\x76') in log:
        idx = log.index(a)
        if idx + 1 < len(log):
            nxt = log[idx + 1] & b'\x0f'[0]
            if nxt == b'\x0c'[0]:
                print(i, idx, nxt)

108 269 12


In [18]:
modem_event_logs = {i:log for i, log in enumerate(logs_helper.logs_list) if log[0:1] == b'\x60'}
keys = list(modem_event_logs.keys()); print(len(keys))
k0 = keys[0]
diff = [(k1, k2, k2 - k1) for k1, k2 in zip(keys, keys[1:])]
diff_less100 = [(k1, k2, d) for k1, k2, d in diff if d < 100]; print(len(diff_less100))
diff_less100

269
32


[(41642, 41644, 2),
 (41644, 41646, 2),
 (181842, 181844, 2),
 (498814, 498822, 8),
 (530414, 530449, 35),
 (536651, 536659, 8),
 (548218, 548220, 2),
 (615687, 615689, 2),
 (647405, 647407, 2),
 (647407, 647409, 2),
 (647409, 647411, 2),
 (647411, 647414, 3),
 (647414, 647417, 3),
 (647417, 647419, 2),
 (721319, 721321, 2),
 (738886, 738900, 14),
 (738900, 738901, 1),
 (738901, 738941, 40),
 (738941, 738942, 1),
 (739852, 739854, 2),
 (739854, 739872, 18),
 (739872, 739874, 2),
 (739874, 739925, 51),
 (739925, 739927, 2),
 (763495, 763498, 3),
 (763498, 763517, 19),
 (764343, 764396, 53),
 (766306, 766307, 1),
 (868662, 868670, 8),
 (882125, 882145, 20),
 (933570, 933634, 64),
 (935500, 935576, 76)]

In [32]:
# hangover start logs
ho_start_logs = {}
for k in modem_event_logs.keys():
    if (a := b'\x74') in (log := logs_helper.logs_list[k]):
        id1 = log.index(a)
        if (id2 := id1 + 1) < len(log):
            nxt = log[id2] & b'\x0f'[0]
            if nxt == b'\x0c'[0]:
                print(k, id1, nxt)
                ho_start_logs[k] = log
print(ho_start_logs.keys())

282504 54 12
506072 98 12
522972 4 12
737011 23 12
882125 29 12
dict_keys([282504, 506072, 522972, 737011, 882125])


In [56]:
positive_ids = []
for k in ho_start_logs.keys():
    positive_ids.append([i for i in range(k-100, k)])
for ids in positive_ids:
    print(f'{ids[0]}, {ids[1]}, {ids[2]}, ..., {ids[-2]}, {ids[-1]}')

282404, 282405, 282406, ..., 282502, 282503
505972, 505973, 505974, ..., 506070, 506071
522872, 522873, 522874, ..., 522970, 522971
736911, 736912, 736913, ..., 737009, 737010
882025, 882026, 882027, ..., 882123, 882124


In [69]:
for ids in positive_ids:
    print(max([len(logs_helper.logs_list[i]) for i in ids]))

1162
2073
1156
1322
1750


In [63]:
neg_start, neg_delta = 40000, 40000
negative_logs_keys = [k for k in range(neg_start, neg_start + 5 * neg_delta, neg_delta)]
negative_ids = []
for k in negative_logs_keys:
    negative_ids.append([i for i in range(k-100, k)])
for ids in negative_ids:
    print(f'{ids[0]}, {ids[1]}, {ids[2]}, ..., {ids[-2]}, {ids[-1]}')

39900, 39901, 39902, ..., 39998, 39999
79900, 79901, 79902, ..., 79998, 79999
119900, 119901, 119902, ..., 119998, 119999
159900, 159901, 159902, ..., 159998, 159999
199900, 199901, 199902, ..., 199998, 199999


In [68]:
for ids in negative_ids:
    print(max([len(logs_helper.logs_list[i]) for i in ids]))

1114
1738
1626
234
2743
